In [1]:
import net
import torch
import os
from face_alignment import align
import numpy as np
from tqdm import tqdm
from matlab_cp2tform import get_similarity_transform_for_cv2
import cv2

In [2]:
adaface_models = {
    'ir_101':"./models/backbonesadaface_ir101_webface12m.ckpt",
}

def load_pretrained_model(architecture='ir_101'):
    # load model and pretrained statedict
    assert architecture in adaface_models.keys()
    model = net.build_model(architecture)
    statedict = torch.load(adaface_models[architecture])['state_dict']
    model_statedict = {key[6:]:val for key, val in statedict.items() if key.startswith('model.')}
    model.load_state_dict(model_statedict)
    model.to('cuda')
    model.eval()
    return model
model = load_pretrained_model()


In [3]:
races = ['African','Asian','Caucasian','Indian']
genders = ['Man','Woman']

In [4]:
African_wom = os.listdir('../images/test/data/African/Woman')
Caucasian_wom = os.listdir('../images/test/data/Caucasian/Woman')
Asian_wom = os.listdir('../images/test/data/Asian/Woman')
Indian_wom = os.listdir('../images/test/data/Indian/Woman')
Asian_man = os.listdir('../images/test/data/Asian/Man')
African_man = os.listdir('../images/test/data/African/Man')
Caucasian_man = os.listdir('../images/test/data/Caucasian/Man')
Indian_man = os.listdir('../images/test/data/Indian/Man')

In [5]:
def to_input(pil_rgb_image):
    np_img = np.array(pil_rgb_image)
    brg_img = ((np_img/ 255.) - 0.5) / 0.5
    tensor = torch.tensor(np.array([brg_img.transpose(2,0,1)])).float()
    return tensor

In [6]:
def alignment(src_img,src_pts):
    ref_pts = [ [30.2946, 51.6963],[65.5318, 51.5014],
        [48.0252, 71.7366],[33.5493, 92.3655],[62.7299, 92.2041] ]
    crop_size = (96, 112)
    src_pts = np.array(src_pts).reshape(5,2)

    s = np.array(src_pts).astype(np.float32)
    r = np.array(ref_pts).astype(np.float32)

    tfm = get_similarity_transform_for_cv2(s, r)
    face_img = cv2.warpAffine(src_img, tfm, crop_size)
    return face_img

In [7]:
cache = {}

In [8]:
for race in races:
    landmark = {}
    with open(f'../images/test/txts/{race}/{race}_lmk.txt') as f:
        landmark_lines = f.readlines()
    for line in landmark_lines:
        l = line.strip().split()
        landmark[l[0]] = [float(k) for k in l[2:]]
    with open(f"../images/test/txts/{race}/{race}_pairs.txt", 'r') as f:
        pairs = f.readlines()
    pairs = [pair.strip().split() for pair in pairs]
    for pair in tqdm(pairs):
        try:
            race2 = None
            gen2 = None
            if pair[0] in eval(f'{race}_wom'):
                    gender = "Woman"
            else:
                    gender = "Man"
            sim = 0
            img1 = f"../images/test/data/Just_Images/{race}/{gender}/{pair[0]}_000{pair[1]}.jpg"
            if len(pair) == 3:
                img2 = f"../images/test/data/Just_Images/{race}/{gender}/{pair[0]}_000{pair[2]}.jpg"
                sim = 1
                name1 = pair[0]+'/'+pair[0]+'_'+'{:04}.jpg'.format(int(pair[1]))
                name2 = pair[0]+'/'+pair[0]+'_'+'{:04}.jpg'.format(int(pair[2]))
            else:
                if pair[2] in African_wom or pair[2] in Caucasian_wom or pair[2] in Asian_wom or pair[2] in Indian_wom:
                    gen2 = "Woman"
                    if pair[2] in African_wom:
                        race2 = "African"
                    elif pair[2] in Caucasian_wom:
                        race2 = "Caucasian"
                    elif pair[2] in Asian_wom:
                        race2 = "Asian"
                    else:
                        race2 = "Indian"
                else:
                    gen2 = "Man"
                    if pair[2] in African_man:
                        race2 = "African"
                    elif pair[2] in Caucasian_man:
                        race2 = "Caucasian"
                    elif pair[2] in Asian_man:
                        race2 = "Asian"
                    else:
                        race2 = "Indian"
                img2 = f"../images/test/data/Just_Images/{race2}/{gen2}/{pair[2]}_000{pair[3]}.jpg"
                name1 = pair[0]+'/'+pair[0]+'_'+'{:04}.jpg'.format(int(pair[1]))
                name2 = pair[2]+'/'+pair[2]+'_'+'{:04}.jpg'.format(int(pair[3]))
            if img1 in cache:
                 vec1 = cache[img1]
            else:
                with torch.no_grad():
                    vec1 = model(torch.tensor(np.expand_dims(cv2.resize(alignment(cv2.imread(img1),landmark[f"/test/data/{race}/{name1}"]),(112,112)).transpose(2,0,1),axis = 0)).float().to('cuda'))
                vec1 = vec1[0].cpu().detach().numpy()
                vec1/=np.linalg.norm(vec1)
                cache[img1] = vec1
            if img2 in cache:
                vec2 = cache[img2]
            else:
                with torch.no_grad():
                    vec2 = model(torch.tensor(np.expand_dims(cv2.resize(alignment(cv2.imread(img2),landmark[f"/test/data/{race}/{name2}"]),(112,112)).transpose(2,0,1),axis = 0)).float().to('cuda'))
                vec2 = vec2[0].cpu().detach().numpy()          
                vec2/=np.linalg.norm(vec2)
                cache[img2] = vec2
            if img1 not in cache:
                    cache[img1] = vec1
            if img2 not in cache:
                    cache[img2] = vec2
            score = np.dot(vec1, vec2.T)[0][0]
            with open(f"./lmks/{race}_lmks.csv", 'a') as f:
                f.write(f"{img1},{img2},{score},{sim}\n")
        except Exception as e:
            print(e)
            continue

  0%|          | 0/6000 [00:00<?, ?it/s]/home/dysl-ai/Desktop/Results-on-RFW/AdaFace/matlab_cp2tform.py:50: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  r, _, _, _ = lstsq(X, U)
 25%|██▌       | 1523/6000 [01:59<04:53, 15.27it/s]

OpenCV(4.10.0) /io/opencv/modules/imgproc/src/imgwarp.cpp:2825: error: (-215:Assertion failed) src.cols > 0 && src.rows > 0 in function 'warpAffine'



100%|██████████| 6000/6000 [07:02<00:00, 14.20it/s]
